<center> <h1> Optimization Laboratory 4: Constrained optimization: equality constraints </h1> </center> 

<center> <h3> Alfons Cordoba / Mateusz Dorobek / Emilio Tylson Baixauli </h3> </center> 


## Experiments of Sequetinal Quadratic Programing

### 1 - Newton step with fixed starting point

The first experiment consist in appling the Sequential Quadratic Optimization 

In [1]:
import numpy as np

In [2]:
def grad(x):
    return np.array( [3 * np.exp(3 * x[0]) - 2 * x[2] * x[0], -4 * np.exp(-4 * x[1]) - 2 * x[2] * x[1], 
                      - x[0] ** 2 - x[1] ** 2 + 1])

def hessian(x):
    row_0 = np.array([9 * np.exp(3 * x[0]) - 2 * x[2], 0, -2 * x[0]]).reshape(3, 1)
    row_1 = np.array([0, 16 * np.exp(-4 * x[1]) - 2 * x[2], -2 * x[1]]).reshape(3, 1)
    row_2 = np.array([-2 * x[0], -2 * x[1], 0]).reshape(3, 1)
    return np.concatenate((row_0, row_1, row_2), axis=1)



def rh_gen(F):
    def rh_update(z, n):
        return -F(z)

    return rh_update

def hess_lag_gen(H):
    
    def hess_lag_update(z, n):
        return H(z)
    
    return hess_lag_update


def step(z, rh, hess_lag, rh_update, hess_lag_update, n, alpha=1, epsilon=1e-10):
    dz = np.linalg.solve(hess_lag, rh)
    z = z + alpha * dz
    rh = rh_update(z, n)
    hess_lag = hess_lag_update(z, n)
    eps_condition = np.linalg.norm(rh[:2]) > epsilon
    return eps_condition, z, rh, hess_lag


def loop(z, n=None):
    eps_condition = True
    niter = 1000
    i_loop = 0
    rh_update = rh_gen(grad)
    hess_lag_update = hess_lag_gen(hessian)
    rh = rh_update(z, n)
    hess_lag = hess_lag_update(z, n)
    while eps_condition and i_loop < niter:
        eps_condition, z, rh, hess_lag = step(z, rh, hess_lag, rh_update, hess_lag_update, n)
        i_loop += 1
    return z






In [10]:
z0 = np.array([-1, 1, -1])
original = np.array([-0.74834, 0.66332, -0.21233])
print("The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)")
res = loop(z0)
print("The algorithm obtained ", res)
error = np.linalg.norm(original - res)
print("Error", error)

The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797494772209388e-06


The obtained solution is approximatly the same as the therical solution.

### 2 - Newton step with random starting point

The second experiment anlyses different starting points, and the convergence to the theorical solution.

In [13]:
np.random.seed(0)
test_starting_points = np.random.uniform(-3, 3, (20, 3))
max_err = 0
for test_starting_point in test_starting_points:
    print("*******")
    print("Starting point ",test_starting_point[:2])
    res = loop(test_starting_point)
    print("The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)")
    print("The algorithm obtained ", res)
    error = np.linalg.norm(original - res)
    print("Error", error)
    print("*******")
    print()
    max_err = error if error > max_err else max_err
print("Max error ", max_err)

*******
Starting point  [0.29288102 1.2911362 ]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797486603500676e-06
*******

*******
Starting point  [ 0.2692991 -0.4580712]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [ 0.91041323 -0.41370006 25.2938552 ]
Error 25.58274649409391
*******

*******
Starting point  [-0.37447673  2.350638  ]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797501908791518e-06
*******

*******
Starting point  [-0.69935089  1.75035023]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797486603574388e-06
*******

*******
Starting point  [0.40826737 2.55357983]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.2

/Users/emiliotylson/.local/share/virtualenvs/Optimization-n0N5CbKl/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/emiliotylson/.local/share/virtualenvs/Optimization-n0N5CbKl/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


By evaluating 20 random starting points, we concluded the algorithm is highly sensitive to them. We have obtained errors of 109.39. 
As the algortih is stating on points close to the solution, the Lagrangian not be convex in those points. Thus, the algorithm diverges form the solution. Therefore, it is important to apply a startegy that can approximate the starting point for a correct convergence to the solution.

### 3 - Merit function

In this experiment we apply the merit that helps to find an approximate close point to the solution so the Sequetinal Quadratic can be applyed.

In [28]:
def f(x):
    return np.exp(3 * x[0]) + np.exp(-4 * x[1])

def grad_f(x):
    return np.array([3*np.exp(3 * x[0]), -4 * np.exp(-4 * x[1])])
    
def h(x):
    return x[0]**2 + x[1]**2 - 1

def grad_h(x):
    return np.array([2*x[0], 2*x[1]])

def merit_function(x, p=10):
    return  f(x)+ p *(h(x))**2

def grad_merit_function(x, p=10):
    return grad_f(x) + p * 2 * h(x) * grad_h(x)

In [29]:
#GRADIENT DESCENT WITH ADAPTATIVE ALPHA AND FIXED STEP
def grad_descent(f,grad_f,x0,fdif_threshold = 1e-20,grad_threshold = 1e-20,alfa_threshold = 1e-20):

    x1 = x0
    f1 = f(x1)
    
    x_list= []
    x_list.append(x1)
    gradf_list = []
    
    loop_out = True
    loop_alfa = True
    j=0
    print ('x0:',x1,'f1:',f1,end = ' ')
    while loop_out:
        j=j+1
        gradf = grad_f(x1)
        gradf_list.append(gradf)
        if np.linalg.norm(gradf) <= grad_threshold:
            print('grad_threshold:', np.linalg.norm(gradf),'x2:',x2,'loops:',j)
            return x2, np.array(x_list), np.array(gradf_list)
        
        alfa = 1.0
        i=0
        while loop_alfa:
            i=i+1
            if alfa < alfa_threshold:
                x_list.append(x2)
                print('alfa_threshold:',alfa,'x2:',x2,'inner loops:',i,'outer loops:',j)
                return x2, np.array(x_list), np.array(gradf_list)
            else:
                x2 = x1 - alfa*gradf
                f2 = f(x2)
                if f2 < f1:
                    loop_alfa = False
                else:
                    alfa = alfa/2
        loop_alfa = True  
        x_list.append(x2)
        if abs(f2-f1) <= fdif_threshold:
            print('fdif_threshold:',abs(f2-f1),'x2:',x2,'loops:',j)
            return x2, np.array(x_list), np.array(gradf_list)

        x1 = x2
        f1 = f2

In [32]:
#Merit function approximation
x0 = [3,2]
x, x_list,_ = grad_descent(merit_function,grad_merit_function,x0)
x

x0: [3, 2] f1: 9543.084263038012 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242933  0.6665431 ] inner loops: 68 outer loops: 367


/Users/emiliotylson/.local/share/virtualenvs/Optimization-n0N5CbKl/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


array([-0.75242933,  0.6665431 ])

In [41]:
np.random.seed(0)
test_starting_points = np.random.uniform(-3, 3, (20, 3))
max_err = 0
acum_error = []
for test_starting_point in test_starting_points:
    print("*******")
    print("Starting point ",test_starting_point[:2])
    res, x_list,_ = grad_descent(merit_function,grad_merit_function, test_starting_point[:2])
    print("The original solution is x = (−0.74834, 0.66332) ")
    print("The merit function optimization algorithm obtained ", res)
    error = np.linalg.norm(original[:2] - res)
    print("Error", error)
    print("*******")
    print()
    max_err = acum_error.append(error) 
    
acum_error = np.array(acum_error)


*******
Starting point  [0.29288102 1.2911362 ]
x0: [0.29288102 1.2911362 ] f1: 8.08060488598412 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242932  0.66654311] inner loops: 68 outer loops: 353
The original solution is x = (−0.74834, 0.66332) 
The merit function optimization algorithm obtained  [-0.75242932  0.66654311]
Error 0.005206822864816334
*******

*******
Starting point  [ 0.2692991 -0.4580712]
x0: [ 0.2692991 -0.4580712] f1: 13.641530029602688 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242936  0.66654307] inner loops: 68 outer loops: 354
The original solution is x = (−0.74834, 0.66332) 
The merit function optimization algorithm obtained  [-0.75242936  0.66654307]
Error 0.005206826134534426
*******

*******
Starting point  [-0.37447673  2.350638  ]
x0: [-0.37447673  2.350638  ] f1: 218.0157821904091 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242931  0.66654312] inner loops: 68 outer loops: 345
The original solution is x = (−0.74834, 0.66332) 
The merit function op

/Users/emiliotylson/.local/share/virtualenvs/Optimization-n0N5CbKl/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


alfa_threshold: 6.776263578034403e-21 x2: [-0.75242935  0.66654308] inner loops: 68 outer loops: 334
The original solution is x = (−0.74834, 0.66332) 
The merit function optimization algorithm obtained  [-0.75242935  0.66654308]
Error 0.005206824843739598
*******

*******
Starting point  [1.66894051 2.22007289]
x0: [1.66894051 2.22007289] f1: 600.218676389356 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242935  0.66654308] inner loops: 68 outer loops: 363
The original solution is x = (−0.74834, 0.66332) 
The merit function optimization algorithm obtained  [-0.75242935  0.66654308]
Error 0.005206824354922578
*******

*******
Starting point  [ 1.79495139 -0.23112383]
x0: [ 1.79495139 -0.23112383] f1: 272.3673686804759 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242932  0.66654311] inner loops: 68 outer loops: 375
The original solution is x = (−0.74834, 0.66332) 
The merit function optimization algorithm obtained  [-0.75242932  0.66654311]
Error 0.005206822906889159
*******

******

In [42]:
print("Max error ", acum_error.max())
print("Mean error ", acum_error.mean())

Max error  0.005206826968786757
Mean error  0.005206823774538808


After starting with 20 random points, we can obtained a mean error of 0.0052. Therefore we conclude from the experment that the merit function can approximate the point to one near to the solution.

### 4 - Finding better starting point

As the merit funtion finds a close point to the solution, we proceed on combining the merit function to find a starting point for the sequential quadratic method.

In [43]:
np.random.seed(0)
test_starting_points = np.random.uniform(-3, 3, (20, 3))
acum_error = []
for test_starting_point in test_starting_points:
    print("*******")
    print("Starting point ",test_starting_point[:2])
    test_starting_point[:2], x_list,_ = grad_descent(merit_function,grad_merit_function, test_starting_point[:2])
    print("Merit starting point ", test_starting_point)
    res = loop(test_starting_point)
    print("The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)")
    print("The algorithm obtained ", res)
    error = np.linalg.norm(original - res)
    print("Error", error)
    print("*******")
    print()
    acum_error.append(error)
acum_error = np.array([acum_error])


*******
Starting point  [0.29288102 1.2911362 ]
x0: [0.29288102 1.2911362 ] f1: 8.08060488598412 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242932  0.66654311] inner loops: 68 outer loops: 353
Merit starting point  [-0.75242932  0.66654311  0.61658026]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797486603595066e-06
*******

*******
Starting point  [ 0.2692991 -0.4580712]
x0: [ 0.2692991 -0.4580712] f1: 13.641530029602688 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242936  0.66654307] inner loops: 68 outer loops: 354
Merit starting point  [-0.75242936  0.66654307  0.87536468]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797493912058464e-06
*******

*******
Starting point  [-0.37447673  2.350638  ]
x0: [-0.37447673  2.350638  ] f1: 218.0157821904091 alfa_threshold: 6.776263578034403e

/Users/emiliotylson/.local/share/virtualenvs/Optimization-n0N5CbKl/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Merit starting point  [-0.75242935  0.66654308  1.99571907]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797499019277177e-06
*******

*******
Starting point  [1.66894051 2.22007289]
x0: [1.66894051 2.22007289] f1: 600.218676389356 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242935  0.66654308] inner loops: 68 outer loops: 363
Merit starting point  [-0.75242935  0.66654308  2.87171005]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)
The algorithm obtained  [-0.74833549  0.66332043 -0.21232494]
Error 6.797486603553708e-06
*******

*******
Starting point  [ 1.79495139 -0.23112383]
x0: [ 1.79495139 -0.23112383] f1: 272.3673686804759 alfa_threshold: 6.776263578034403e-21 x2: [-0.75242932  0.66654311] inner loops: 68 outer loops: 375
Merit starting point  [-0.75242932  0.66654311  1.68317506]
The theorical solution is x = (−0.74834, 0.66332) and lambda = (−0.21233)

In [44]:
print("Mean error ", acum_error.mean())

Mean error  6.797489558207493e-06


After starting the full algoith with different starting points we see the combination of the two algorithms mangaed to find the solution. Comparing to previous result, this method obtained the best convergence of all. The merit function optimization method alone obtained a point with an error around 0.0052, while the combination with the sequential quadratic method obtained an error of 6.8 e-6. This is a critical improvement of the convergence to the solution.

## Conclusion

The sequential quadratic method is an interative method that allows the optimization of constrained functions by using second order information of the Lagrangian function. Despite converging to the solution, this convergence strongly depends on the starting point. This starting point should be near the solution so the method takes advantage on the convexity of the function. In order to find better starting points, the laboratory proposes the merit function optimization. This function penalises points outside the feasible set. By minimizaing this function, it can be obtained an approximation that is close to the solution but could be outside the feasible set. Then, by using this point as a starting point of the afformentioned sequential quadratic method, we can build a method for optimizing contrained function that not depends on the starting point.